In [49]:
import os
from deta import Deta
from dotenv import load_dotenv
import sqlite3
import pandas as pd

### Loading DB key

In [2]:
load_dotenv(".env")
DETA_KEY = os.getenv("DETA_KEY")

### Initializing DB key

In [3]:
deta = Deta(DETA_KEY)

### Initializing DB

In [4]:
db = deta.Base("exp_tracker_app")

### DB functions

In [5]:
def insert_period(period, incomes, expenses, comment):
    """Returns monthly report"""
    return db.put({"key": period, "incomes": incomes, "expenses": expenses, "comment": comment})

def fetch_all_periods():
    """Returns a dict of all periods"""
    res = db.fetch()
    return res.items

def get_period(period):
    return db.get(period)

### Example data

In [6]:
period = "2022_May"
comment = "init"
incomes = {'Salary': 1500, 'Dividend': 500, 'Invest Return': 100}
expenses = {'Food': 200, 'Loan': 300, 'CC Bill': 30}

#### DB inserting example data

In [7]:
insert_period(period, incomes, expenses, comment)

{'comment': 'init',
 'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
 'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
 'key': '2022_May'}

#### Fetching all data from DB

In [8]:
fetch_all_periods()

[{'comment': 'init',
  'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
  'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
  'key': '2022_May'},
 {'comment': 'first entry',
  'expenses': {'CC Bill': 30, 'Food': 40, 'Loan': 170, 'Others': 200},
  'incomes': {'Dividend': 10, 'Invest Return': 20, 'Salary': 450},
  'key': '2023_January'}]

#### Fetching single period data from DB

In [9]:
get_period("2022_May")

{'comment': 'init',
 'expenses': {'CC Bill': 30, 'Food': 200, 'Loan': 300},
 'incomes': {'Dividend': 500, 'Invest Return': 100, 'Salary': 1500},
 'key': '2022_May'}

In [44]:
db2 = deta.Base("wallet_db")

In [40]:
def insert_trans(period, date, _type, amount, wallet_id, category_id, transfer_wallet_id, trans_amount, subcategory_id, note):
    """Inserts daily transaction"""
    return db2.put({'key': period, 'date': date, 'type': _type, 'amount': amount, 'wallet_id': wallet_id, 'category_id': category_id, 'transfer_wallet_id': transfer_wallet_id, 'trans_amount': trans_amount, 'subcategory_id': subcategory_id, 'note': note})

def fetch_all_trans():
    """Returns a dict of all periods"""
    res = db2.fetch()
    return res.items

# def get_period(period):
#     """Returns monthly report"""
#     return db.get(period)

In [39]:
insert_trans('Jun-23', '24/06/2023', 'inc', 10, 1, 2, -1, 0, 0, 'test')

{'amount': 10,
 'category_id': 2,
 'date': '24/06/2023',
 'key': 'Jun-23',
 'note': 'test',
 'subcategory_id': 0,
 'trans_amount': 0,
 'transfer_wallet_id': -1,
 'type': 'inc',
 'wallet_id': 1}

In [45]:
fetch_all_trans()

[{'amount': 10,
  'category_id': 2,
  'date': '24/06/2023',
  'key': 'Jun-23',
  'note': 'test',
  'subcategory_id': 0,
  'trans_amount': 0,
  'transfer_wallet_id': -1,
  'type': 'inc',
  'wallet_id': 1}]

In [50]:
dbfile = 'ignore/wallet-database.db'
con = sqlite3.connect(dbfile)
cur = con.cursor()
transaction_df = pd.read_sql_query('SELECT * FROM trans', con)          #? all transactions table
con.close()
df = transaction_df.copy()
df['amount'] = df['amount'].div(100).round(2)
df['trans_amount'] = df['trans_amount'].div(100).round(2)
df['type'] = df['type'].replace([0, 1, 2], ['inc', 'exp', 'tran'])
df.drop(['id', 'fee_id', 'account_id', 'debt_id', 'debt_trans_id', 'memo'], axis = 1, inplace = True)
df['date_time'] = pd.to_datetime(df["date_time"], unit='ms').dt.date
df = df[['date_time', 'type', 'amount', 'wallet_id', 'category_id', 'transfer_wallet_id', 'trans_amount', 'subcategory_id', 'note']]
df

,date_time,type,amount,wallet_id,category_id,transfer_wallet_id,trans_amount,subcategory_id,note
0,2020-08-31,exp,-50.0,1,30,-1,0.0,0,
1,2020-09-02,inc,25.0,2,34,-1,0.0,0,
2,2020-09-02,exp,-50.0,5,30,-1,0.0,0,
3,2020-09-02,exp,-49.0,2,32,-1,0.0,0,mama chhoto
4,2020-09-02,inc,50.0,1,33,-1,0.0,0,mama chhoto
...,...,...,...,...,...,...,...,...,...
2249,2023-06-07,exp,-24.0,11,6,-1,0.0,0,canteen
2250,2023-06-08,exp,-146.0,11,11,-1,0.0,0,me+dk - ezee35+basmatiRice65+aloo10+haldi36
2251,2023-06-09,exp,-32.0,11,6,-1,0.0,0,canteen
2252,2023-06-10,exp,-20.0,11,6,-1,0.0,0,me+dk-fuchka


In [58]:
year = df.iloc[0,0].strftime("%Y")
print("year:", year)

month = df.iloc[0,0].strftime("%b")
print("month:", month)

year: 2020
month: Aug
